# Process Dataset

The Yelp Dataset can be downloaded [here](https://www.yelp.com/dataset). 

The Yelp Dataset claims to have 8,635,403 reviews. To reduce the scale of this project, I will select a subset of the total dataset. 

Here is a [useful guide](https://towardsdatascience.com/load-yelp-reviews-or-other-huge-json-files-with-ease-ad804c2f1537) on avoiding reading the entire dataset into memory. 



In [1]:
import pandas as pd 

business_dataset_path = "yelp_dataset/yelp_academic_dataset_business.json"
review_dataset_path = "yelp_dataset/yelp_academic_dataset_review.json"
user_dataset_path = "yelp_dataset/yelp_academic_dataset_user.json"

## Select subset based on business location

Selecting a subset of reviews for businesses located in California sounds like a good selecting.

Or so I thought. It turns out that only certain locations are included in this Yelp Dataset, and California only has 13 businesses included. 

I guess I could have discovered that earlier by reading the FAQ. 

> How many and which cities are included in the dataset?
> 
> Currently, the metropolitan areas centered on Montreal, Calgary, Toronto, Pittsburgh, Charlotte, Urbana-Champaign, Phoenix, Las Vegas, Madison, and Cleveland, are included in the dataset.

In [2]:
# The Business dataset is relatively small. Let's start by loading that entire dataset into memory, so we can select the businesses located in California. 

# Read json into memory
business_df = pd.read_json(business_dataset_path, orient="records", lines=True)
# Keep only features of interest
business_columns_to_keep = ["business_id", "state", "stars", "review_count", "is_open"]
business_df = business_df[business_columns_to_keep]

print(f"Business Datset has {len(business_df)} total entries.")

business_df["state"].value_counts()

Business Datset has 160585 total entries.


MA     36012
OR     25175
TX     24485
FL     21907
GA     18090
BC     17298
OH     11258
CO      3198
WA      3121
CA        13
NH         4
AZ         2
NY         2
VA         2
ON         2
WY         1
OK         1
AL         1
ME         1
NC         1
DE         1
MI         1
ABE        1
MN         1
KS         1
DC         1
KY         1
IL         1
HI         1
NM         1
WI         1
Name: state, dtype: int64

## Select subset based on review date

Since it appears that the dataset only includes businesses in certain locations, perhaps I can select reviews based on the date. 

Below I read through the entire review dataset to get counts of the number of reviews per year. 

It seems that the reviews are pretty comprehensive, and the dataset was assembled sometime in early 2021. 

Now that we know the number of reviews per year, let's select only reviews written in 2019. This selection would be about 1/8 the total dataset. 

| Year | Number of Reviews |
|------|-------------------|
| 2004 | 52                |
| 2005 | 6439              |
| 2006 | 23819             |
| 2007 | 71916             |
| 2008 | 150436            |
| 2009 | 213797            |
| 2010 | 317583            |
| 2011 | 431192            |
| 2012 | 472441            |
| 2013 | 555740            |
| 2014 | 726119            |
| 2015 | 907529            |
| 2016 | 960527            |
| 2017 | 1029557           |
| 2018 | 1084335           |
| 2019 | 1037569           |
| 2020 | 601891            |
| 2021 | 44461             |

In [3]:
# Read through the reviews dataset, selecting only reviews written in 2019

# Get counts of the number of reviews written in each year
review_years = []
review_df = []
# Only select reviews with a date in this set of years
selected_years = {2019}
# Only keep relevent features to reduce the size of our dataset
review_columns_to_keep = ["text", "stars", "date"]

# Read the JSON object into a JsonReader to iterate chunk by chunk
reader = pd.read_json(review_dataset_path, orient="records", lines=True, chunksize=1000)

i = 1
for chunk in reader:
	# There are about 8,636,000 reviews.
	# With chunksize=1000, there are about 8,636 chunks.
	print(f"Reading chunk {i} of 8636", end="\r")
	i += 1

	# Extract the year from each review date. 
	# This section was used to count the number of reviews per year
	year = chunk['date'].apply(lambda x: x.year)
	review_years.append(year)
	
	# Keep only reviews where the date has a year in selected_years.
	selected_values = chunk[review_columns_to_keep]
	selected_values = selected_values[selected_values["date"].dt.year.isin(selected_years)]
	review_df.append(selected_values)

print("\nMerging selected entries into DataFrame...")
# Make df of selected entries
review_years = pd.concat(review_years, ignore_index=True)
review_df = pd.concat(review_df, ignore_index=True)

# Get count of reviews per year
review_years = review_years.value_counts().sort_index()

# Drop date from review_df, not needed for NLP
review_df.drop(columns=["date"], inplace=True)
# Save selected entries to disk
print("Saving selected reviews to disk...")
review_df.to_json("yelp_dataset/reviews.json", orient="records", lines=True)
print("Review subset saved to disk.")

Reading chunk 8636 of 8636
Merging selected entries into DataFrame...
Saving selected reviews to disk...
Review subset saved to disk.


In [4]:
# Display count of reviews per year
review_years

2004         52
2005       6439
2006      23819
2007      71916
2008     150436
2009     213797
2010     317583
2011     431192
2012     472441
2013     555740
2014     726119
2015     907529
2016     960527
2017    1029557
2018    1084335
2019    1037569
2020     601891
2021      44461
Name: date, dtype: int64